# Imports

In [ ]:
import numpy as np
import pandas as pd

import mne
from mne.io import read_raw_edf
from mne.filter import notch_filter, filter_data

# Config

In [ ]:
class Config:
    def __init__(self, data_path='/Users/guillaumecorda/Desktop/UvA/Big Data/Project/data/'):
        self.data_path = data_path

In [ ]:
cfg = Config()

# Metadata

**File Name: chb01_01.edf**
* File Start Time: 11:42:54
* File End Time: 12:42:54
* Number of Seizures in File: 0

**File Name: chb01_02.edf**
* File Start Time: 12:42:57
* File End Time: 13:42:57
* Number of Seizures in File: 0

**File Name: chb01_03.edf**
* File Start Time: 13:43:04
* File End Time: 14:43:04
* Number of Seizures in File: 1
* Seizure Start Time: 2996 seconds
* Seizure End Time: 3036 seconds

**File Name: chb01_04.edf**
* File Start Time: 14:43:12
* File End Time: 15:43:12
* Number of Seizures in File: 1
* Seizure Start Time: 1467 seconds
* Seizure End Time: 1494 seconds

# Load data

In [ ]:
raw_1 = read_raw_edf(cfg.data_path+'chb01_01.edf', preload=True)
raw_2 = read_raw_edf(cfg.data_path+'chb01_02.edf', preload=True)
raw_3 = read_raw_edf(cfg.data_path+'chb01_03.edf', preload=True)
raw_4 = read_raw_edf(cfg.data_path+'chb01_04.edf', preload=True)

In [ ]:
raw_1.set_eeg_reference('average', projection=True)  # set EEG average reference
raw_2.set_eeg_reference('average', projection=True)  # set EEG average reference
raw_3.set_eeg_reference('average', projection=True)  # set EEG average reference
raw_4.set_eeg_reference('average', projection=True)  # set EEG average reference

In [ ]:
raw_1.plot(block=True, lowpass=40)

In [ ]:
for raw in [raw_1, raw_2, raw_3, raw_4]:
    fmin, fmax = 2, 300  # look at frequencies between 2 and 300Hz
    n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2

    picks = mne.pick_types(raw.info, eeg=True, eog=False, stim=False, exclude='bads')

    # Let's first check out all channel types
    raw.plot_psd(area_mode='range', tmax=10.0, picks=picks)

# Helper functions

In [ ]:
def get_sampl_freq(raw):
    return raw.info['sfreq']

In [ ]:
def get_channels_names(raw):
    return raw.info['ch_names']

In [ ]:
sf_1 = get_sampl_freq(raw_1)
sf_2 = get_sampl_freq(raw_2)
sf_3 = get_sampl_freq(raw_3)
sf_4 = get_sampl_freq(raw_4)

# Preprocessing

## Removing power line frequency

In [ ]:
filtered_data = notch_filter(raw.get_data(), sf_1, np.arange(48, 52, 4000), picks=picks, filter_length='auto', phase='zero')
raw_1_filt = mne.io.RawArray(filtered_data, raw_1.info)

raw_1.plot_psd(area_mode='range', tmax=10.0, picks=picks)
raw_1_filt.plot_psd(area_mode='range', tmax=10.0, picks=picks)

## Keeping only low frequencies

In [ ]:
filtered_data = filter_data(raw_1_filt.get_data(), sf_1, l_freq=2, h_freq=60, picks=picks, filter_length='auto', phase='zero')
raw_1_filt = mne.io.RawArray(filtered_data, raw_1.info)
raw_1_filt.plot_psd(area_mode='range', tmax=10.0, picks=picks, fmax=60)